In [1]:
from __future__ import division
import matplotlib.pyplot as plt
import pandas
import pdb, traceback, sys
import scipy
from scipy import signal
import datetime

# Imports useful for graphics
import matplotlib.pyplot as plt
import seaborn
seaborn.set_style("whitegrid")
seaborn.despine()
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}

# Give you access to all the V2G-Sim modules
import h2vgi

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [ ]:
# Create a project that will hold other objects such as vehicles, locations
# car models, charging stations and some results. (see model.Project class)
project = h2vgi.model.Project()

# Use the itinerary module to import itineraries from an Excel file.
# Instantiate a project with the necessary information to run a simulation.
# Default values are assumed for the vehicle to model
# and the charging infrastructures to simulate.
project = h2vgi.itinerary.from_excel(project, '../data/NHTS/Tennessee.xlsx')
nb_of_days = 40
project = h2vgi.itinerary.copy_append(project, nb_of_days_to_add=nb_of_days)

for vehicle in project.vehicles:
    vehicle.result_function = h2vgi.result.save_vehicle_power_demand

# Launch the simulation and save the results
h2vgi.core.run(project, date_from=project.date, date_to=project.date +  datetime.timedelta(days=nb_of_days))

itinerary.from_excel(project, ../data/NHTS/Tennessee.xlsx)
Loading ...


In [ ]:
# total cumsumption
total_consumption = project.locations[0].result
for location in project.locations[1:]:
    total_consumption += location.result

In [ ]:
plt.figure(figsize=(12, 5), dpi=60)
plt.plot(total_consumption.power_demand.cumsum() * 60)
plt.title('Hydrogen consumption at the pump for ' + str(len(project.vehicles)) + ' vehicles')
plt.ylabel('Total consumption [kg]')
plt.xlabel('Time')
plt.show()

In [ ]:
# Hydrogen plant characteristics
full_tank = 500  # [kg]
initial_tank = 500  # [kg]
low_level = 0.2  # without units
efficiency = 1.3  # without units
hydrogen_production = 0.01  # [kg/s]
hydrogen_to_watt = 39  # [kWh/kg]

# http://www.renewableenergyfocus.com/view/3157/hydrogen-production-from-renewables/
generator_on = False
tank_level = [initial_tank]
power_consumption = []
on = []
vehicle_consumption = total_consumption.power_demand.tolist()  # [kg/s] every minutes

# Simulate the hydrogen station cycles
for consumed in vehicle_consumption:
    # Append consumption
    tank_level.append(tank_level[-1] - consumed * 60)  # same power consumption within the minute
    
    # If the level is too low and the generator is not running
    if (tank_level[-1] / full_tank <= low_level) and not generator_on:
        # Start the generator
        generator_on = True
    else:
        # Do nothing
        pass
    
    # Save generator state
    on.append(generator_on)
    
    # If the generator is on -> increase tank level
    if generator_on:
        tank_level[-1] += hydrogen_production * 60  # [kg/s]
        # [kg/h] * [kWh/kg] = kW 
        power_consumption.append(hydrogen_production * 3600 * hydrogen_to_watt * efficiency)
        
        # If the full level is reached turn generator off
        if tank_level[-1] >= full_tank:
            generator_on = False
    
    # If generator is off there is no electric consumption
    else:
        power_consumption.append(0.0)

# Remove initial value?
del tank_level[0]

# Create frame to gather all the results
station = pandas.DataFrame(index=total_consumption.index, data={'power': power_consumption,
                                                                'tank': tank_level,
                                                                'on': on})

In [ ]:
plt.figure(figsize=(11, 5), dpi=60)
plt.plot(station.power)
plt.ylabel('Power consumption [kW]')
plt.xlabel('Time')
plt.show()

In [ ]:
plt.figure(figsize=(11, 5), dpi=60)
plt.plot(station.tank)
plt.ylabel('Tank level [kg]')
plt.xlabel('Time')
plt.show()

In [ ]:
duration = []
period = []
already_on = False
minute_to_hour = 1 / 60

# duration distribution
for value in station.itertuples():
    if value.on and not already_on:
        # Start the count
        already_on = True
        temp_count = 0
    
    if not value.on and already_on:
        already_on = False
        duration.append(temp_count * minute_to_hour)
    
    if already_on:
        temp_count += 1

already_off = False
# frequency distribution
for value in station.itertuples():
    if not value.on and not already_off:
        # Start the count
        already_off = True
        temp_count = 0
    
    if value.on and already_off:
        already_off = False
        period.append(temp_count * minute_to_hour)
    
    if already_off:
        temp_count += 1
# Remove first one
del period[0]

In [ ]:
plt.figure(figsize=(11, 4), dpi=60)
plt.scatter([i for i in range(1, len(duration)+1)], duration)
plt.ylabel('Duration of a station on cycle [hours]')
plt.xlabel('Cycle id')
plt.show()

In [ ]:
plt.figure(figsize=(11, 4), dpi=60)
plt.scatter([i for i in range(1, len(period)+1)], period)
plt.ylabel('Period between station turning on [hours]')
plt.xlabel('Cycle id')
plt.show()

In [ ]:
print('Utilization factor: ' + str(station[station.on == True].on.count() * 100 / len(station)) + ' %')